<a href="https://colab.research.google.com/github/bxck75/colab_projects/blob/master/Cartoonize_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Pick a filesystem
filesystem = "colab" #@param ["colab", "drive"]

In [0]:
# Pick a style
styles = "all" #@param ["shinkai", "hayao", "hosoda", "paprika","all"]

In [0]:
# pick source 
source = 'upload' #@param ["upload","url"]

# Prepair

## set filesystem

In [4]:
if filesystem == 'drive':
  # sync google drive
  from google.colab import drive
  import os
  # if drive is needed uncomment
  print('Google drive ',end='')
  drive.mount('/content/drive',force_remount=True)
  print('Root folder set to ',end='')
  root = '/content/drive/My drive/Cartoonizer'
  print(root)
else:
  print('Root folder set to ',end='')
  root = '/content/Cartoonizer'
  print(root)

Root folder set to /content/Cartoonizer


## import needed stuff

In [0]:
# imports
import time,os,glob,cv2, itertools
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output, display, Image
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
!apt install tree


repo = "CartoonGan-tensorflow"
!git clone https://github.com/mnicnc404/{repo}.git
os.chdir(os.path.join(repo))
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install imageio
!git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install
  
# from subprocess import call
import tensorflow as tf
tf.__version__

#function defines
def resize(image,window_height = 1000):
    aspect_ratio = float(image.shape[1])/float(image.shape[0])
    window_width = window_height/aspect_ratio
    image = cv2.resize(image, (int(window_height),int(window_width)))
    return image

def download_image(url,input_file_name):
  !curl -o {input_file_name} {url}
  img = cv2.imread(input_file_name, cv2.IMREAD_UNCHANGED)
  return img

# clear the long output
clear_output()

## Set I/O

In [0]:
# set i/o
input_folder = 'input'
output_folder = 'output'
# check if paths are in place
if not os.path.exists(root):
  os.mkdir(root)
  os.mkdir(root+'/'+output_folder)  
  os.mkdir(root+'/'+input_folder)
  

## Pauze to upload images to input folder

In [0]:
input( "upload the image to " + root+'/'+input_folder )

## Resize

In [0]:
if source == 'url':
  # Set the img url
  img_url = input('Enter the image url : ')
  # extract the name and add the style
  img_name = img_url.split('.')
  img_ext = img_name[len(img_name)-1]
  img_name = img_name[len(img_name)-2]
  img_name = img_name.split('/')[2]
  img_name = img_name + '_' + styles +'.'+ img_ext
  print(img_in_folder+img_name)
  img_array = download_image(img_url,img_in_folder+img_name)

In [0]:
# Set i/o folders
img_in_folder =  root+'/'+input_folder
img_out_folder = root+'/'+output_folder

In [0]:
# resize if needed
max_size=1000
searchedfiles = glob.glob(img_in_folder+'/*.jpg')
files = sorted( searchedfiles, key = lambda file: os.path.getctime(file))
# for file in files check height
for f in files:
  img = cv2.imread(f)
  h,w,c = img.shape
  # Resize if needed
  if h > max_size:
    img_resized = resize(img,window_height = max_size)
    rh,rw,rc = img_resized.shape
    cv2.imwrite(f,img_resized)

# Cartoon time

In [0]:
def Run():
  
  !python cartoonize.py \
    --comparison_view grid \
    --keep_original_size \
    --batch_size 4 \
    --styles '{styles}' \
    --input_dir '{img_in_folder}' \
    --output_dir '{img_out_folder}'
  !mv '{img_out_folder}/comparison' '{img_out_folder}/{styles}/comparison'
  
if styles == 'all':
  print('Looping over all styles')
  for styles in ["shinkai", "hayao", "hosoda", "paprika"]:
    Run()    
else:
  print('Running with '+styles+" style")
  !python cartoonize.py \
    --comparison_view horizontal \
    --keep_original_size \
    --batch_size 4 \
    --styles '{styles}' \
    --input_dir '{img_in_folder}' \
    --output_dir '{img_out_folder}'  
  !mv '{img_out_folder}/comparison' '{img_out_folder}/{styles}/comparison' 

## Make grid

In [0]:
import math
import os
import matplotlib.pyplot as plt
images_list= []
# Config:
images_dir = '/content/Cartoonizer/output'
result_grid_filename = '/content/Cartoonizer/output/grid.jpg'
result_figsize_resolution = 50 # 1 = 100px

for roots, dirs, files in os.walk(images_dir):
    for file in files:
        if file.endswith('.jpg'):
          images_list.append(roots+'/'+file)

# images_list = os.listdir(images_dir)
images_count = len(images_list)
# print('Images: ', images_list)
print('Images count: ', images_count)

# Calculate the grid size:
grid_size = math.ceil(math.sqrt(images_count))

# Create plt plot:
fig, axes = plt.subplots(grid_size, grid_size, figsize=(result_figsize_resolution, result_figsize_resolution))

current_file_number = 0
for image_filename in images_list:
  if image_filename != 'grid.jpg':
    x_position = current_file_number % grid_size
    y_position = current_file_number // grid_size

    plt_image = plt.imread(images_list[current_file_number])
    axes[x_position, y_position].imshow(plt_image)
    print((current_file_number + 1), '/', images_count, ': ', image_filename)

    current_file_number += 1

plt.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=1.0)
plt.savefig(result_grid_filename)

## Zip

In [0]:
!zip -r /content/file.zip {img_out_folder}